# Create ISA-API Investigation from Datascriptor Study Design configuration

In this notebook I will show you how you can use a study design configuration is JSON format as produce by datascriptor (https://gitlab.com/datascriptor/datascriptor) to generate a single-study ISA investigation and how you can then serialise it in JSON and tabular (i.e. CSV) format.

Or study design configuration consists of:
- a 6-arm study design. Each arm has 10 subjects
- there is an observational factor with 3 values, which is age_group (young, middle-aged, elderly)
- a crossover of two treatments, a drug treatment and a biological treatment
- three non-treatment phases: screen, washout and follow-up
- two sample types colllected: blood and saliva
- two assay types: 
    - metabolite profiling through mass spectrometry on the saliva sample. The mass spec will be run on a "Agilent QTQF 6510" instrument, testing both "FIA" and "LC" injection modes, and "positive" acquisition mode.
    - metabolite profiling through  NMR spectroscopy on the blood samples.  The NMR will be run on a "Bruker Avance II 1 GHz" instrument, on "1D 1H NMR" acquisition mode, testing both "CPGM" amd "TOCSY" pulse sequences.

## 1. Setup

Let's import all the required libraries

In [1]:
from time import time
import os
import json

In [2]:
## ISA-API related imports
from isatools.model import Investigation, Study

In [3]:
## ISA-API create mode related imports
from isatools.create.model import StudyDesign
from isatools.create.connectors import generate_study_design_from_config

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

In [4]:
## ISA-API create mode related imports
from isatools.create import model
from isatools import isajson

## 2. Load the Study Design JSON configuration

First of all we load the study design configurator with all the specs defined above

In [5]:
with open(os.path.abspath(os.path.join(
    "config", ""
)), "r") as config_file:
    study_design_config = json.load(config_file)
study_design_config

{'observationalFactors': [{'name': 'age group',
   'values': ['young', 'middle-aged', 'elderly'],
   'isQuantitative': False,
   'unit': None}],
 'generatedSubjectGroups': [{'name': 'SubjectGroup_0',
   'type': 'Homo sapiens',
   'characteristics': [{'name': 'age group',
     'value': 'young',
     'unit': None,
     'isQuantitative': False}]},
  {'name': 'SubjectGroup_1',
   'type': 'Homo sapiens',
   'characteristics': [{'name': 'age group',
     'value': 'middle-aged',
     'unit': None,
     'isQuantitative': False}]},
  {'name': 'SubjectGroup_2',
   'type': 'Homo sapiens',
   'characteristics': [{'name': 'age group',
     'value': 'elderly',
     'unit': None,
     'isQuantitative': False}]}],
 'selectedSubjectGroups': [{'name': 'SubjectGroup_0',
   'type': 'Homo sapiens',
   'characteristics': [{'name': 'age group',
     'value': 'young',
     'unit': None,
     'isQuantitative': False}]},
  {'name': 'SubjectGroup_1',
   'type': 'Homo sapiens',
   'characteristics': [{'name': 'ag

## 3. Generate the ISA Study Design from the JSON configuration
To perform the conversion we just need to use the function `generate_isa_study_design_from_config` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [6]:
study_design = generate_study_design_from_config(study_design_config)
assert isinstance(study_design, StudyDesign)

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [7]:
start = time()
study = study_design.generate_isa_study()
end = time()
print('The generation of the study design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)
investigation = Investigation(studies=[study])

The generation of the study design took 5.59 s.


## 5. Serialize and save the JSON representation of the generated ISA Investigation

In [8]:
start = time()
inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
end = time()
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the ISA investigation took 1.61 s.


In [9]:
directory = os.path.abspath(os.path.join('output'))
if not os.path.exists(directory):
    os.makedirs(directory)
with open(os.path.abspath(os.path.join('output','isa-investigation-2-arms-nmr-ms.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Dump the ISA Investigation to ISA-Tab

In [10]:
start = time()
isatab.dump(investigation, os.path.abspath(os.path.join('output')))
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The Tab serialisation of the ISA investigation took 92.73 s.


To use them on the notebook we can also dump the tables to pandas DataFrames, using the `dump_tables_to_dataframes` function rather than dump

In [11]:
dataframes = isatab.dump_tables_to_dataframes(investigation)

In [12]:
len(dataframes)

3

## 7. Check the correctness of the ISA-Tab DataFrames 

We have 1 study file and 2 assay files (one for MS and one for NMR). Let's check the names:

In [13]:
for key in dataframes.keys():
    display(key)

's_study_01.txt'

'a_AT2_metabolite-profiling_NMR-spectroscopy.txt'

'a_AT1_metabolite-profiling_mass-spectrometry.txt'

### 7.1 Count of subjects and samples

We have 10 subjects in the each of the six arms for a total of 60 subjects. 5 blood samples per subject are collected (1 in treatment 1 phase, 1 in treatment, and 3 in the follow-up phase) for a total of 300 blood samples. These will undergo the NMR assay. We have 4 saliva samples per subject (1 during screen and 3 during follow-up) for a total of 240 saliva samples. These will undergo the "mass spcetrometry" assay.

In [14]:
study_frame = dataframes['s_study_01.txt']
count_arm0_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP0' in el)])
count_arm2_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP1' in el)])
count_arm3_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP2' in el)])
count_arm4_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP3' in el)])
count_arm3_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP4' in el)])
count_arm4_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP5' in el)])
print("There are {} samples in the GRP0 arm (i.e. group)".format(count_arm0_samples))
print("There are {} samples in the GRP2 arm (i.e. group)".format(count_arm2_samples))
print("There are {} samples in the GRP3 arm (i.e. group)".format(count_arm3_samples))
print("There are {} samples in the GRP4 arm (i.e. group)".format(count_arm4_samples))

There are 90 samples in the GRP0 arm (i.e. group)
There are 90 samples in the GRP2 arm (i.e. group)
There are 90 samples in the GRP3 arm (i.e. group)
There are 90 samples in the GRP4 arm (i.e. group)


In [15]:
dataframes['a_AT1_metabolite-profiling_mass-spectrometry.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Term Accession Number,Protocol REF.1,Performer.1,Labeled Extract Name,Label,Protocol REF.2,Parameter Value[instrument],Term Accession Number.1,Parameter Value[injection_mode],Term Accession Number.2,Parameter Value[acquisition_mode],Term Accession Number.3,MS Assay Name,Performer.2,Raw Spectral Data File
0,GRP0_SBJ01_A0E0_SMP-saliva-1,extraction,Unknown,AT1-S9-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S9-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S9-mass-spectrometry-Acquisition-R1,Unknown,AT1-S9-raw-spectral-data-file-R1
1,GRP0_SBJ01_A0E0_SMP-saliva-1,extraction,Unknown,AT1-S9-Extract-R2,lipids,lipids,labeling,Unknown,AT1-S9-LE-R4,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S9-mass-spectrometry-Acquisition-R15,Unknown,AT1-S9-raw-spectral-data-file-R15
2,GRP0_SBJ01_A0E0_SMP-saliva-1,extraction,Unknown,AT1-S9-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S9-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S9-mass-spectrometry-Acquisition-R5,Unknown,AT1-S9-raw-spectral-data-file-R5
3,GRP0_SBJ01_A0E0_SMP-saliva-1,extraction,Unknown,AT1-S9-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S9-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S9-mass-spectrometry-Acquisition-R8,Unknown,AT1-S9-raw-spectral-data-file-R8
4,GRP0_SBJ01_A0E0_SMP-saliva-1,extraction,Unknown,AT1-S9-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S9-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S9-mass-spectrometry-Acquisition-R7,Unknown,AT1-S9-raw-spectral-data-file-R7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3835,GRP5_SBJ10_A5E4_SMP-saliva-3,extraction,Unknown,AT1-S237-Extract-R2,lipids,lipids,labeling,Unknown,AT1-S237-LE-R3,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S237-mass-spectrometry-Acquisition-R11,Unknown,AT1-S237-raw-spectral-data-file-R11
3836,GRP5_SBJ10_A5E4_SMP-saliva-3,extraction,Unknown,AT1-S237-Extract-R2,lipids,lipids,labeling,Unknown,AT1-S237-LE-R3,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S237-mass-spectrometry-Acquisition-R12,Unknown,AT1-S237-raw-spectral-data-file-R12
3837,GRP5_SBJ10_A5E4_SMP-saliva-3,extraction,Unknown,AT1-S237-Extract-R2,lipids,lipids,labeling,Unknown,AT1-S237-LE-R3,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S237-mass-spectrometry-Acquisition-R10,Unknown,AT1-S237-raw-spectral-data-file-R10
3838,GRP5_SBJ10_A5E4_SMP-saliva-3,extraction,Unknown,AT1-S237-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S237-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S237-mass-spectrometry-Acquisition-R2,Unknown,AT1-S237-raw-spectral-data-file-R2


In [16]:
dataframes['a_AT2_metabolite-profiling_NMR-spectroscopy.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Parameter Value[instrument],Parameter Value[acquisition_mode],Parameter Value[pulse_sequence],Performer.1,Free Induction Decay Data File
0,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT2-S17-Extract-R2,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,Unknown,AT2-S17-raw_spectral_data_file-R7
1,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT2-S18-Extract-R1,supernatant,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,Unknown,AT2-S18-raw_spectral_data_file-R3
2,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT2-S18-Extract-R2,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,TOCSY,Unknown,AT2-S18-raw_spectral_data_file-R6
3,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT2-S18-Extract-R2,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,Unknown,AT2-S18-raw_spectral_data_file-R8
4,GRP0_SBJ01_A0E1_SMP-blood-1,extraction,Unknown,AT2-S18-Extract-R1,supernatant,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,TOCSY,Unknown,AT2-S18-raw_spectral_data_file-R2
...,...,...,...,...,...,...,...,...,...,...,...
4795,GRP5_SBJ10_A5E4_SMP-blood-3,extraction,Unknown,AT2-S593-Extract-R1,supernatant,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,Unknown,AT2-S593-raw_spectral_data_file-R3
4796,GRP5_SBJ10_A5E4_SMP-blood-3,extraction,Unknown,AT2-S593-Extract-R2,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,TOCSY,Unknown,AT2-S593-raw_spectral_data_file-R5
4797,GRP5_SBJ10_A5E4_SMP-blood-3,extraction,Unknown,AT2-S594-Extract-R2,pellet,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,Unknown,AT2-S594-raw_spectral_data_file-R7
4798,GRP5_SBJ10_A5E4_SMP-blood-3,extraction,Unknown,AT2-S594-Extract-R1,supernatant,nmr_spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,Unknown,AT2-S594-raw_spectral_data_file-R4


### 7.1 Overview of the Mass Spec assay table

For the mass. spec. assay table, we have 240 (saliva) samples, 480 extracts (2 per  sample, "lipids" and "polar" fractions), 960 labeled extracts (2 per extract, as "#replicates" is  2) and 3840 mass spec protocols + 3840 output files (4 per labeled extract as we do 2 technical replicates with 2 protocol parameter combinations `["Agilent QTQF 6510", "FIA", "positive mode"]` and `["Agilent QTQF 6510", "LC", "positive mode"]`).

In [19]:
dataframes['a_AT1_metabolite-profiling_mass-spectrometry.txt'].nunique(axis=0, dropna=True)

Sample Name                           240
Protocol REF                            1
Performer                               1
Extract Name                          480
Characteristics[extract type]           2
Term Accession Number                   2
Protocol REF.1                          1
Performer.1                             1
Labeled Extract Name                  960
Label                                   1
Protocol REF.2                          1
Parameter Value[instrument]             1
Term Accession Number.1                 1
Parameter Value[injection_mode]         2
Term Accession Number.2                 2
Parameter Value[acquisition_mode]       1
Term Accession Number.3                 1
MS Assay Name                        3840
Performer.2                             1
Raw Spectral Data File               3840
dtype: int64

### Overview of the NMR assay table

For the NMR assay table, we have 300 (blood) samples, 1200 extracts (4 per  sample, 2 extraction replicates of the "supernatant" and "pellet" fractions) and 4800 NMR protocols + 4800 output files (4 per extract as we do 2 technical replicates with 2 protocol parameter combinations `["Bruker Avance II 1 GHz", "1D 1H NMR", "CPGM"]` and `["Bruker Avance II 1 GHz", "1D 1H NMR", "TOCSY"]`).

In [20]:
dataframes['a_AT2_metabolite-profiling_NMR-spectroscopy.txt'].nunique(axis=0, dropna=True)

Sample Name                           300
Protocol REF                            1
Performer                               1
Extract Name                         1200
Characteristics[extract type]           2
Protocol REF.1                          1
Parameter Value[instrument]             1
Parameter Value[acquisition_mode]       1
Parameter Value[pulse_sequence]         2
Performer.1                             1
Free Induction Decay Data File       4800
dtype: int64